In [1]:
import pandas as pd
import numpy as np
import json
import ast
import gzip

In [17]:

df_games = pd.read_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_games.csv')

In [4]:
df_items = pd.read_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_items.csv')

In [5]:

df_review = pd.read_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_reviews.csv')

In [12]:
def PlayTimeGenre(genero: str, df_games: pd.DataFrame, df_items: pd.DataFrame):
    """""
    Devuelve año con mas horas jugadas para dicho género.
    """
    
    # Verificar si el género está presente como una columna en df_games
    if genero not in df_games.columns:
        return f"No se encontró información para el género {genero}"

    # Filtrar el df de juegos para obtener los juegos que pertenecen al género específico
    juegos_genero = df_games[df_games[genero] == 1]

    if juegos_genero.empty:
        return f"No se encontró información para el género {genero}"

    # Obtener los nombres de los juegos para el género específico
    nombres_juegos_genero = juegos_genero['app_name']

    # Filtrar el df de horas jugadas para obtener las horas correspondientes a esos nombres de juegos
    horas_jugadas_genero = df_items[df_items['item_name'].isin(nombres_juegos_genero)]

    # Convertir la columna 'id' a int64 si es posible
    df_games['id'] = pd.to_numeric(df_games['id'], errors='coerce')

    # Fusionar los df después de convertir las columnas al mismo tipo
    horas_jugadas_genero = horas_jugadas_genero.merge(df_games[['id', 'año']], left_on='item_id', right_on='id')

    # Encontrar el año con más horas jugadas para el género específico
    año_mas_horas = horas_jugadas_genero.groupby('año')['playtime_forever'].sum().idxmax()

    return {f"Año de lanzamiento con más horas jugadas para el género {genero}": año_mas_horas}

In [13]:
PlayTimeGenre('Action', df_games, df_items)

{'Año de lanzamiento con más horas jugadas para el género Action': 2012.0}

In [24]:
def UserForGenre(genero: str, df_games: pd.DataFrame, df_items: pd.DataFrame):
    """""
    Devuelve el usuario que acumula más horas jugadas para el género dado 
    y una lista de la acumulación de horas jugadas por año."""
    
    # Filtrar juegos por género
    juegos_genero = df_games[df_games[genero] == 1]
    if juegos_genero.empty:
        return f"No se encontró información para el género {genero}"

    # Obtener nombres de juegos y filtrar horas jugadas por género
    nombres_juegos_genero = juegos_genero['app_name']
    horas_jugadas_genero = df_items[df_items['item_name'].isin(nombres_juegos_genero)]

    # Fusionar df y encontrar usuario con más horas jugadas
    merged_data = horas_jugadas_genero.merge(df_games[['id', 'año']], left_on='item_id', right_on='id')
    user_mas_horas = merged_data.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Calcular horas jugadas por año del usuario
    horas_por_año = merged_data[merged_data['user_id'] == user_mas_horas].groupby('año')['playtime_forever'].sum().to_dict()

    return user_mas_horas, horas_por_año

In [26]:
UserForGenre('Action', df_games, df_items)


('DeEggMeister',
 {1994.0: 1.0,
  1995.0: 80.0,
  1996.0: 0.0,
  1997.0: 0.0,
  1998.0: 1504.0,
  1999.0: 993.0,
  2000.0: 7276.0,
  2001.0: 295.0,
  2002.0: 1388.0,
  2003.0: 4364.0,
  2004.0: 62077.0,
  2005.0: 5118.0,
  2006.0: 6855.0,
  2007.0: 20012.0,
  2008.0: 94711.0,
  2009.0: 185133.0,
  2010.0: 103453.0,
  2011.0: 296094.0,
  2012.0: 1113909.0,
  2013.0: 434469.0,
  2014.0: 130637.0,
  2015.0: 202919.0,
  2016.0: 89277.0,
  2017.0: 87156.0})

In [18]:
def UsersRecommend(año: int, df_items: pd.DataFrame, df_review: pd.DataFrame):
    """""
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado
    basandose en reviews.recommend = True y comentarios positivos/neutrales
    """

    # Filtrar las reviews para el año dado, recomendadas y con análisis de sentimiento bueno o neutral
    filtered_reviews = df_review[(df_review['año_publicado'].notnull()) & 
                                 (df_review['año_publicado'] == año) & 
                                 (df_review['recommend'] == True) & 
                                 (df_review['sentiment_analysis']== 2) | (df_review['sentiment_analysis'] == 1)]
                                  

    # Obtener los user_ids de las reviews filtradas
    user_ids_recomendados = filtered_reviews['user_id']

    # Filtrar los items jugados por los usuarios recomendados
    items_jugados_recomendados = df_items[df_items['user_id'].isin(user_ids_recomendados)]

    # Obtener los nombres de los juegos más jugados por los usuarios recomendados
    top_games = items_jugados_recomendados['item_name'].value_counts().head(3)

    #Itera sobre top_games sumandole un valor al idice (comienza en 0) para devolver el puesto y el juego
    resultado = [{"Puesto " + str(i + 1): juego} for i, juego in enumerate(top_games.index)]
    return resultado


In [19]:
UsersRecommend(2014, df_items, df_review)

[{'Puesto 1': 'Dota 2 Test'},
 {'Puesto 2': 'Counter-Strike: Global Offensive'},
 {'Puesto 3': "Garry's Mod"}]

In [27]:
def UsersNotRecommend(año: int, df_items: pd.DataFrame, df_review: pd.DataFrame):
    """""
    Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.
    basandose en reviews.recommend = False y comentarios negativos
    """
    
    # Filtrar las reviews para el año dado, no recomendadas y con análisis de sentimiento negativo
    filtered_bad_reviews = df_review[(df_review['año_publicado'].notnull()) & 
                                     (df_review['año_publicado'] == año) & 
                                     (df_review['recommend'] == False) & 
                                     (df_review['sentiment_analysis'] == 0)]

    # Obtener los user_ids de las reviews filtradas
    user_ids_not_recommended = filtered_bad_reviews['user_id']

    # Filtrar los items jugados por los usuarios no recomendados
    items_jugados_no_recomendados = df_items[df_items['user_id'].isin(user_ids_not_recommended)]

    # Obtener los nombres de los juegos menos jugados por los usuarios no recomendados
    less_games = items_jugados_no_recomendados['item_name'].value_counts().head(3)

    #Itera sobre top_games sumandole un valor al idice (comienza en 0) para devolver el puesto y el juego
    resultado = [{"Puesto " + str(i + 1): juego} for i, juego in enumerate(less_games.index)]
    return resultado


In [28]:
UsersNotRecommend(2014, df_items, df_review)

[{'Puesto 1': 'Dota 2 Test'},
 {'Puesto 2': "Garry's Mod"},
 {'Puesto 3': 'Counter-Strike: Global Offensive'}]

In [29]:
def sentiment_analysis(año: int, df_review: pd.DataFrame):
    """""
    Según el año de lanzamiento, se devuelve una lista con la cantidad 
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
    2=positivo 1=neutral 0=negativo
    """

    # Filtrar las reviews para el año dado, excluyendo los valores nulos en 'año_publicado'
    reviews_year = df_review[(df_review['año_publicado'].notnull()) & (df_review['año_publicado'] == año)]

    # Contar la cantidad de registros por cada categoría de sentimiento para el año dado
    sentiment_counts = reviews_year['sentiment_analysis'].replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).value_counts()

    # Crear el diccionario con la cantidad de registros por categoría de sentimiento
    resultado = {sentiment: sentiment_counts.get(sentiment, 0) for sentiment in ['Negative', 'Neutral', 'Positive']}
    return resultado


In [30]:
sentiment_analysis(2014, df_review)

{'Negative': 4486, 'Neutral': 4757, 'Positive': 12868}